# Molecular diagnosis of Huntington’s Disease in Trinidadian families via Triplet Repeat Primed PCR, Fragment analysis, and Nanopore sequencing

Shavana Nicole Rajkumar<sup>1</sup>, Chris Gyan<sup>2</sup>, Damion Basdeo<sup>2</sup>, Nemal Gokool<sup>2</sup>, Arianne Brown Jordan<sup>3</sup>, Soren Nicholls<sup>3</sup>, Vijay Pradeep<sup>4</sup>, and Rajini Rani Haraksingh<sup>1</sup>

<sup>1</sup>Department of Life Sciences, Faculty of Science and Technology, The University of the West Indies, St. Augustine, Trinidad and Tobago<br>
<sup>2</sup>Sangre Grande Hospital, Eastern Regional Health Authority, Trinidad & Tobago<br>
<sup>3</sup>Department of Preclinical Sciences, Faculty of Medical Sciences, The University of the West Indies, St. Augustine, Trinidad and Tobago<br>
<sup>4</sup>Virtana TT Ltd, St. Augustine, Trinidad and Tobago

## Abstract
**Background:** Huntington’s Disease (HD) is a neurodegenerative disorder caused by a CAG expansion in the Huntingtin (HTT) gene. Due to its non-specific and variable phenotype, diagnosis requires clinical assessments and genetic testing. In the Caribbean, the genetic etiology of HD is underexplored due to the unavailability of genetic testing.

**Objectives:** We investigated whether 32 participants from 4 multigenerational families from Trinidad and Tobago (T&T) presenting with Huntington-like symptoms carried HTT CAG expansions, and whether progressive expansion was related to decreasing age of onset with each generation.

**Results:** All symptomatic participants carried HTT CAG expansions (42-57 CAGs), confirming HD. Among participants aged 20-65 years (n=24), clinical and genetic diagnoses were 100% concordant for 22 participants (13 symptomatic with 42-57 CAGs, and 9 asymptomatic with 13-27 CAGs). Two asymptomatic participants aged 22 and 43 years carried 46-47 and 37-39 CAGs, respectively. Among 8 participants &lt;18 years, one symptomatic 16 year old carried 49-50 CAGs, and 7 are currently asymptomatic (3 with 50-52 CAGs, and 4 with 14-17 CAGs). In all families, progressive expansion and decreasing age of onset was observed in each successive generation. Methods were highly correlated (r2 = 0.998).

**Conclusions:** We demonstrated the novel application of nanopore sequencing with a custom bioinformatics workflow to accurately size HTT CAG repeats. This is the first genetic report of HD in T&T, among limited records in the Caribbean.

Link to Publication: *(TODO)*


# Running the Google Colab Informatics Pipeline:
* Load this notebook into Google Colab ([or follow this link](https://colab.research.google.com/drive/1MIvhA1dgq093tx29I-hkY7WooaI4Mn25))
* Find files barcode01.fastq.gz to barcode23.fastq.gz on your local computer.
* In the toolpane on the left side, find the 'files' option
* Create a folder in the colab called 'input'
* Upload all *.fastq.gz files into the input folder
* Run the script (Go to runtime->run all)
* Text output is printed inline. Graphs are printed inline, and also saved as individual files within the colab instance (available via the file browser in the lefthand toolbar).


# Install Dependencies
 * [fastq](https://github.com/not-a-feature/fastq) - Used to parse the fastq input files
 * [swalign](https://github.com/mbreese/swalign) - Used for doing Smith-Waterman style local alignment
 * [miniFasta](https://github.com/not-a-feature/miniFASTA) - Used by fastq

In [ ]:
!pip install fastq==2.0.2 swalign==0.3.7 miniFasta==3.0.3

# Load cached results, if available

The Smith-Waterman alignment is the most time consuming step in the overall pipeline (Approx 5-10 minutes per sample, depending on the number of reads). As such, this script optionally saves the Smith-Waterman results into a python 'pickle' file, so that they can be easily reloaded back into memory.

To use the previously cached results, they must be uploaded to session storage at `preprocessed/barcodes-pkl-0-24.tgz`, via the colab's file manager, available in the toolbar on the left side of the editor.

You can also download previously cached results, generated by the authors, from the following Google Drive link: [2024-02-22-barcodes-pkl-0-24.tgz](https://drive.google.com/file/d/1DlclntCt__4KTni4jrezbohci8bg3o1h/view?usp=drive_link)

In [ ]:
import os
if os.path.exists('preprocessed/2024-02-22-barcodes-pkl-0-24.tgz'):
  !tar xzf preprocessed/2024-02-22-barcodes-pkl-0-24.tgz -C preprocessed/


# Choose what samples to process

While testing the pipeline or experimenting with alternative implementations, it can be helpful to adjust this section to only 1 or 2 samples to save time.

In [ ]:
sample_names = ['barcode%02i' % x for x in range(1,25)]
print(sample_names)


# Run alignment on all samples

For each sample, the fastq reads are loaded and parsed. Forward and reverse start and end primers are aligned to each read using two methods:
 * **Regular Expressions** - This approach only works when there is a perfect match. This approach is only used for debugging purposes and to help sanity-check that Smith-Waterman alignment is operating as expected.
 * **Smith-Waterman Alignment** - This allows for imperfect matches. Matches are scored as +1 for every match, and -1 for every mismatch.

 The matching results for each sample are stored in the `Preprocessed` object, which can be used for further analysis.

 Each `Preprocessed` object is also stored to disk as a [python pickle](https://docs.python.org/3/library/pickle.html) file (`.pkl`) to serve as a cache.  If the pickle file is found on disk, then these results are loaded instead of rerunning the alignment.

 Reads should be uploaded to `input/barcode[i].fastq.gz` for i=00,01...24

In [ ]:
import fastq as fq
import swalign
import os
import types
import re
import pickle

class PreprocessedSample:
    def __init__(self, sample_name):
        self.sample_name = sample_name
        self.raw_reads = None

    def load_file(self, filename=None, max_lines=None):
        if filename:
            self.filename = filename
        else:
            self.filename = os.path.join('input', self.sample_name + ".fastq.gz")
        print("-------")
        print(f"Processing File '{self.filename}'")
        raw_reads_gen = fq.read(self.filename)
        self.raw_reads = list(raw_reads_gen)  # Convert to list for simplicity
        print(f"Loaded {len(self.raw_reads)} reads")
        if max_lines and len(self.raw_reads) > max_lines:
            print(f"Trimming reads down to {max_lines}")
            self.raw_reads = self.raw_reads[:max_lines]

    def preprocess_re(self, cur_primers):
        print(f"{self.sample_name} - Performing RegEx Matching:")
        self.re = types.SimpleNamespace()
        self.re.raw = {}
        # Loop over each primer pair, and rerun search
        for primer_name, primer_vals in cur_primers.items():
            assert len(primer_vals) == 2
            self.re.raw[primer_name] = [None, None]
            for i, primer_val in enumerate(primer_vals):
                print(f"  Processing {primer_name} - {i}")
                raw_re_match = [re.search(primer_val, x.body) for x in self.raw_reads]
                self.re.raw[primer_name][i] = [ x.span() if x else None for x in raw_re_match]
                count = len([x for x in self.re.raw[primer_name][i] if x is not None])
                print(f"    Got {count} matches")

    def preprocess_sw(self, cur_primers):
        print(f"{self.sample_name} - Performing Smith-Waterman Matching:")
        self.sw = types.SimpleNamespace()
        sw = swalign.LocalAlignment(scoring_matrix=swalign.IdentityScoringMatrix(match=1, mismatch=-1))
        # Loop over each primer pair, and rerun search
        self.sw.raw = {}
        for primer_name, primer_vals in cur_primers.items():
            assert len(primer_vals) == 2
            self.sw.raw[primer_name] = [None, None]
            for i, primer_val in enumerate(primer_vals):
                print(f"  Processing {primer_name} - {i}")
                self.sw.raw[primer_name][i] = [ sw.align(primer_val, x.body) for x in self.raw_reads ]
                count_0_errors = len([x for x in self.sw.raw[primer_name][i] if x.score == len(primer_val)])
                print(f"    Got {count_0_errors} matches with 0 errors")

primers = dict()
primers['f53'] = ('ATGAAGGCCTTCGAGTCCCTCAAGTCC','CAGCAGCAGCAGCAGCAACAGCCGCCACCG')
primers['r53'] = ('CGGTGGCGGCTGTTGCTGCTGCTGCTGCTG','GGACTTGAGGGACTCGAAGGCCTTCAT')

pickle_dir = 'preprocessed'
if not os.path.exists(pickle_dir):
    os.makedirs(pickle_dir)

preprocessed_samples = dict()
for sample_name in sample_names:
    pickle_file = os.path.join(pickle_dir, sample_name + '.pkl')
    if os.path.isfile(pickle_file):
        print(f'Loading {sample_name} from pkl')
        with open(pickle_file, 'rb') as f:
            sample = pickle.load(f)
    else:
        print(f'Loading {sample_name} from raw reads')
        sample = PreprocessedSample(sample_name)
        sample.load_file(max_lines=None)
        sample.preprocess_re(primers)
        sample.preprocess_sw(primers)

        # Save to pickle
        print(f'Saving {sample_name} to pickle')
        with open(pickle_file, 'wb') as f:
            pickle.dump(sample, f)
    print(f"  {len(sample.raw_reads)} Reads Found")
    preprocessed_samples[sample_name] = sample


# Compute Read Lengths

The read length (i.e. the # of nucleotides between the start and end primer) for each is calculated, based on various criteria.  A read is considered valid if both of the following criteria are met:
* The start primer and end primer both have an acceptable alignment matching score.
* The start primer appears before the end primer in the read

This calculation is repeated for regular expression alignment, and then Smith-Waterman, with the acceptable matching score being incrementally loosened from 0 mismatches all the way to 10 mismatches.

Additional diagnostic information is printed for both forward and reverse primers to aid in debugging:
* *Start* - The number of reads where the start primer was found
* *End* - The number of reads where the end primer was found
* *Both* - The number of reads where both the start & end primer were found
* *Seq* - The number of reads where the start & end primer were found sequentially (i.e. start comes before end).  These are the only reads where a read length was successfully computed.

Note: As the number of allowed Smith-Waterman matches increases, it's possible to get an increase in spurious matches. In some extreme cases, a single read will have start and end primer matches for both the forward and reverse primers, which is biochemically nonsensical. This is flagged as an error and this specific read is ignored.

In [ ]:
class GenericMatches:
    def __init__(self, sample_name, match_method, generic_matches):
        self.sample_name     = sample_name
        self.match_method    = match_method
        self.generic_matches = generic_matches

    def compute_lengths(self):
        def match_length(start, end):
            if (start is None or end is None):
                return -1
            match_len = end[0] - start[1] - 1
            if match_len < 0:
                return -2
            return match_len

        self.lengths_per_primer = dict()
        total_reads = len(list(self.generic_matches.values())[0][0])
        print(f"{self.sample_name} - Post-Processing '{self.match_method}' Matches.  {total_reads} reads")
        for primer_name, match_lists in self.generic_matches.items():
            lengths = [ match_length(start, end) for start, end in zip(match_lists[0], match_lists[1]) ]
            num_start = sum( [1 for x in match_lists[0] if x] )
            num_end   = sum( [1 for x in match_lists[1] if x] )
            num_both  = sum( [1 for x,y in zip(match_lists[0], match_lists[1]) if x and y])
            num_seq   = sum( [1 for x in lengths if x >= 0])
            print(f"  {primer_name}:  Start:{num_start:5}  End:{num_end:5}  Both:{num_both:5}  Seq:{num_seq:5}")
            self.lengths_per_primer[primer_name] = lengths
        # Compute Combined Length
        self.lengths = []
        for i,length_tuple in enumerate(zip(*self.lengths_per_primer.values())):
            valid_count = sum( (1 for x in length_tuple if x >= 0) )
            if valid_count == 0:
                pass
            elif valid_count == 1:
                self.lengths.append(next((x for x in length_tuple if x >= 0)) )
            else:
                print(f"Error: Detected multiple primer matches ({self.sample_name}-{i})")
        print(f"  Combined: {len(self.lengths)}")

def build_generic_re_matches(preprocessed_sample):
    generic_matches = dict()
    for primer_name, matches in preprocessed_sample.re.raw.items():
        generic_matches[primer_name] = [ matches[0], matches[1] ]
    return GenericMatches(preprocessed_sample.sample_name, 're', generic_matches)

def build_generic_sw_matches(preprocessed_sample, max_errors):
    generic_matches = dict()
    for primer_name, matches in preprocessed_sample.sw.raw.items():
        start_matches = [ (x.q_pos, x.q_end) if (x.score >= len(x.ref) - max_errors) else None for x in matches[0] ]
        end_matches   = [ (x.q_pos, x.q_end) if (x.score >= len(x.ref) - max_errors) else None for x in matches[1] ]
        generic_matches[primer_name] = [ start_matches, end_matches ]
    return GenericMatches(preprocessed_sample.sample_name, f"sw-{max_errors}", generic_matches)

post_samples = dict()
for sample_name, processed_sample in preprocessed_samples.items():
    post_samples[sample_name] = types.SimpleNamespace()

    post_samples[sample_name].generic_re = build_generic_re_matches(processed_sample)
    post_samples[sample_name].generic_re.compute_lengths()

    num_sw = 11
    post_samples[sample_name].generic_sw = [None] * num_sw
    for i in range(num_sw):
        post_samples[sample_name].generic_sw[i] = build_generic_sw_matches(processed_sample, i)
        post_samples[sample_name].generic_sw[i].compute_lengths()

# Generate summary text file
This section generates a `.csv` file with a summary of the matching results.
Below are the headers for the output:
* `name` - Sample Name
* `total_reads` - The total # of reads loaded from the sample's fastq file
* `re` - The number of reads with successful primer matches, using regular expression matching
* `sw[i]` i = 0...10 - The number of reads with successful primer matches, using Smith-Waterman alignment, allowing up to [i] mismatches per primer.

In [ ]:
# Create a CSV file with the specified columns
import csv

with open('num_matches_sw.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['name', 'total_reads', 're'] + [f'sw{i}' for i in range(num_sw)])

    # Iterate over the post_samples dictionary and write the data to the CSV file
    for sample_name, post_sample in post_samples.items():
        writer.writerow([
            sample_name,
            len(list(post_sample.generic_re.generic_matches.values())[0][0]),
            len(post_sample.generic_re.lengths),
        ] + [len(post_sample.generic_sw[i].lengths) for i in range(num_sw)])


# Visualize how many reads were successful
This section visualizes the results from the previous csv file. This helps in building intuition around whether or not allowing more mismatches in the Smith-Waterman matching actually helps us recover for reads.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker

# Create a figure with a single subplot
fig, ax = plt.subplots()

# Iterate over the post_samples dictionary and plot the data
for i, (sample_name, post_sample) in enumerate(post_samples.items()):
    total_reads = len(list(post_sample.generic_re.generic_matches.values())[0][0])
    success_rate = [len(post_sample.generic_sw[i].lengths) / total_reads for i in range(num_sw)]
    ax.plot(range(num_sw), success_rate, label=sample_name)

# Add a legend and grid
ax.grid(True)

ax.set_ylim(0, 1)
ax.yaxis.set_major_formatter(matplotlib.ticker.PercentFormatter(1))

# Set the x and y axis labels
ax.set_xlabel('# Allowed Mismatches')
ax.set_ylabel('% of Successful Reads')

# Add a title to the plot
ax.set_title('Success Rate vs Allowed Mismatches')

# Show the plot
plt.show()



# Generate Histograms of Match Lengths
For each alignment approach for each sample, we generate a histogram of read lengths.  These are all saved to disk and aggregated into the file `output/output.tgz`, which can be downloaded using Colab's file browser in the sidebar.

Additional Notes:
* The match length is manually increased by 15 nucleotides, since the a portion of primers themselves also contain nucleotides of interest.
* Match lengths are capped to 300. For example, a read that is 425 long is represented as 300 on the histogram. This helps to maintain consistency across the graphs and avoid generating confusingly wide graphs.

In [ ]:
# Make an output folder for the graphs
try:
  os.mkdir('output')
except OSError as error:
  if error.errno != 17:
    raise
  print("Folder already exists.  Ignoring.")
  pass


In [ ]:
from matplotlib import pyplot as plt
import collections

max_length = max([max(x.generic_re.lengths +
                  [max(y.lengths) for y in x.generic_sw])
                  for x in post_samples.values()])
print(f"Max Match Length is {max_length}")
max_length = 300
print(f"Hardcoding Max Match Length to {max_length}")

for sample_name, sample in post_samples.items():
    for generic_x in [sample.generic_re] + sample.generic_sw:
        plt.figure(figsize=(15,10))
        adj_lengths = [min(x + 15, max_length-1) for x in generic_x.lengths]
        plt.hist(adj_lengths, range(max_length), edgecolor='black')
        plt.xlim(0, max_length)
        plt.title(f'Histogram of Final Match Lengths for {sample_name}-{generic_x.match_method}')
        plt.xlabel('Final Match Length')
        plt.ylabel('Frequency')
        ax = plt.gca()
        ax.grid(which='both')

        tick_locations = range(0, max_length + 1, 20)
        tick_labels = [str(i) for i in tick_locations]
        plt.xticks(tick_locations, tick_labels)

        ax.minorticks_on()
        ax.grid(which='minor', color='gray', linestyle=':', linewidth=0.5)
        ax.set_xticks(range(0, max_length, 5), minor=True)

        counter = collections.Counter(adj_lengths)
        most_common = counter.most_common(6)
        text = '\n'.join(['(%3i,%3i)' % (value, count) for value, count in most_common])
        text = "Peaks:   \n" + text
        plt.text(0.95, 0.95, text, transform=plt.gca().transAxes, fontsize=8,
                verticalalignment='top', horizontalalignment='right',
                fontdict={'family': 'monospace'})

        plt.savefig(f'output/{sample_name}-{generic_x.match_method}.png')
        plt.show()


In [ ]:
# Generate a tarball of all the graphs, to simply downloading the results from the colab
!tar -cvzf output.tgz output

# Example Introspection & Debugging

Since intermediate state is being stored through the entire processing pipeline, it's straightforward to introspect into any specific alignment or read.

In this specific case (FastQ Read #2013 from barcode01), the raw regular expression alignment & Smith-Waterman alignment matching results are being displayed from multiple stages of processing.  In this specific example, the forward starting primer aligns to two different locations, depending on whether regular expression alignment or Smith-Waterman 0-mismatch alingment is used.  In theory both algorithms should return the same alignment. However, upon closer inspection, the the start primer actually appears twice in the read, explaining the irregularity.

In [ ]:
print(post_samples['barcode01'].generic_re.generic_matches['f53'][0][2013])
print(post_samples['barcode01'].generic_sw[0].generic_matches['f53'][0][2013])
print("---")
preprocessed_samples['barcode01'].sw.raw['f53'][0][2013].dump()
print("---")
print(preprocessed_samples['barcode01'].re.raw['f53'][0][2013])
print(preprocessed_samples['barcode01'].raw_reads[2013].body[69:96])
print(preprocessed_samples['barcode01'].raw_reads[2013].body[172:199])
